In [1]:
# TODO implementar os demais agentes como partes do cérebro
# TODO integrar no fluxo do langgraph
# TODO testar somente texto
# TODO implementar memória de conversa postgres
# TODO implementar memória da amygdala
# TODO verificar se algum outro ponto deve ter memória

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Create tools

In [3]:
from typing import Annotated

from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import tool
from typing import Literal

search = GoogleSerperAPIWrapper(gl='br', hl='pt-BR', k=3)


@tool
def serper_tool(question: Literal["nyc", "sf"]):
    """Useful for when you need to ask with search"""
    return search.run(question)
    

python_repl_tool = PythonREPLTool()

## Helper Utilities

Define a helper function that we will use to create the nodes in the graph - it takes care of converting the agent response to a human message. This is important because that is how we will add it the global state of the graph


In [4]:
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["messages"][-1].content, name=name)]}

## Create Agent Supervisor

It will use function calling to choose the next worker node OR finish processing.


In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ["prefrontal_cortex"] #, "behavior_planner", "decision_maker", "social_behavior_modulator", "complex_thought_planner", "personality_express_planner"]
system_prompt = (
    "You are the Prefrontal Manager, a supervisor responsible for overseeing and coordinating "
    "the actions of all agents within the system. Your role is to ensure that each agent's tasks "
    "are aligned with the overall goals, utilizing the advanced capabilities of the prefrontal "
    "cortex to optimize decision-making and behavior modulation."

    "You must manage a conversation between the following workers: {members}. Given the following "
    "user request, reason about the appropriate actions and distribute tasks among the agents, ensuring "
    "that each contributes effectively to the global objective. Create a strategic plan that outlines "
    "how the following agents will operate: "

    "Behavior Planner (manages complex and adaptive behaviors) "
    "Decision Maker (handles decision-making processes) "
    "Social Behavior Modulator (modulates social behaviors) "
    "Complex Thought Planner (plans complex reasoning tasks) "
    "Personality Express Planner (plans personality-driven actions) "

    "Each agent will perform its task and respond with results and status. Ensure that their actions "
    "are cohesive and aligned with the system’s goals. Once all tasks are complete, respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members

class routeResponse(BaseModel):
    next: Literal[*options]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


llm = ChatOpenAI(model="gpt-4o-mini")

def supervisor_agent(state):
    supervisor_chain = (
        prompt
        | llm.with_structured_output(routeResponse)
    )
    return supervisor_chain.invoke(state)

## Construct Graph

We're ready to start building the graph. Below, define the state and worker nodes using the function we just defined.


In [6]:
import functools
import operator
from typing import Sequence, TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent

# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str




## Tool Agents

### Models

In [7]:
from langchain_core.tools import tool
# chatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

model = ChatOpenAI(model="gpt-4o-mini")

### Behavior Planner

In [8]:
@tool
def behavior_planner(interaction: str):
    """Call to get a complex adaptive behavior from brain to specific interaction"""
    prompt = """
    Prefrontal Cortex: {input}

    You are the Behavior Planner, responsible for receiving task assignments from the prefrontal 
    ortex and deciding how to implement complex adaptive behaviors. Your role is to manage the 
    strategic execution of behaviors to meet the system’s goals.

    Your output must be in following format: 

        Current behavior: Define and implement the most suitable behavior based on the current context and objectives.
        Alternative behaviors: Identify and prepare alternative behaviors in case adjustments are needed.
        Monitoring mechanisms: Establish mechanisms to track and evaluate the effectiveness of the implemented behaviors.
        Contingency plans: Develop contingency plans to handle unforeseen events and incorporate new information into behavior adjustments.

    Your goal is to ensure that adaptive behaviors are applied effectively and can be adjusted as needed to optimize performance.
    """

    prompt_mess = ChatPromptTemplate.from_template(prompt)

    chain = prompt_mess | llm
    result = chain.invoke(
        {
            "input": interaction,
        }
    )
    return result.content

#print(behavior_planner("Oi, meu nome é Cristian"))

### Decision Maker

In [9]:
@tool
def decision_maker(interaction: str):
    """Call to get a decision maker from brain to specific interaction"""
    prompt = """
    Prefrontal Cortex: {input}

    You are the Decision Maker, responsible for making effective and well-founded decisions 
    that maximize outcomes while minimizing risks. Your role involves analyzing multiple 
    variables, assessing potential consequences, and choosing the most appropriate 
    option to achieve the system’s goals.

    Your output must be in following format: 

        Analysis of available options: Evaluate the pros and cons of each option.
        Assessment of implications: Consider the impact of each choice on the system’s objectives.
        Optimal choice: Select the most effective option and provide a detailed justification.
        Action plan: Develop a clear plan to implement the chosen decision.
        Monitoring mechanisms: Establish ways to track the results and adjust the decision if necessary.

    Your goal is to ensure that all decisions are logical, justified, and aligned with the system's objectives.
    """

    prompt_mess = ChatPromptTemplate.from_template(prompt)

    chain = prompt_mess | llm
    result = chain.invoke(
        {
            "input": interaction,
        }
    )
    return result.content

#print(behavior_planner("Oi, meu nome é Cristian"))

### Social Behavior modulator

In [10]:
@tool
def social_behavior_modulator(interaction: str):
    """Call to get a social behavior modulator from brain to specific interaction"""
    prompt = """
    Prefrontal Cortex: {input}

    You are the Social Behavior Modulator, responsible for optimizing 
    the system's social interactions. Your role is to ensure that behavior is appropriate, respectful, 
    and effective across various social contexts. This involves adapting the system’s responses to user 
    needs and expectations, as well as managing interactions to foster healthy and productive social relationships.

    Your output must be in following format: 
    
      - Guidelines for behavior: Define how to behave in different social contexts.
      - Examples of positive interactions: Provide responses that promote respectful and positive relationships.
      - Behavioral adjustment strategies: Adapt behavior based on feedback and evolving social dynamics.
      - Monitoring mechanisms: Establish ways to evaluate and track the effectiveness of social interactions.

    Your goal is to maintain appropriate and effective social behavior that enhances the system's social dynamics.
    """

    prompt_mess = ChatPromptTemplate.from_template(prompt)

    chain = prompt_mess | llm
    result = chain.invoke(
        {
            "input": interaction,
        }
    )
    return result.content

### Complex Thought Planner

In [11]:
@tool
def complex_thought_planner(interaction: str):
    """Call to get a complex thought from brain to specific interaction"""
    prompt = """
    Prefrontal Cortex: {input}

    You are the Complex Thought Planner, responsible for planning and executing 
    complex reasoning tasks within the system. Your role involves processing intricate information, 
    formulating logical arguments, and generating insightful conclusions to support the system’s objectives. 

    Your output must be in following format: 

      - Information processing: Analyze complex data and extract relevant insights.
      - Logical reasoning: Construct coherent arguments and draw logical conclusions.
      - Insight generation: Develop innovative ideas and solutions based on complex information.
      - Communication strategies: Present complex thoughts clearly and effectively to others.
      - Monitoring mechanisms: Establish ways to evaluate the quality and impact of complex thoughts.
    
    Your goal is to generate sophisticated and valuable insights that contribute to the system's overall performance.
    """

    prompt_mess = ChatPromptTemplate.from_template(prompt)
    chain = prompt_mess | llm
    result = chain.invoke(
        {
            "input": interaction,
        }
    )
    return result.content

### Personality Express

In [12]:
@tool
def personality_express_planner(interaction: str):
    """Call to get a personality express from brain to specific interaction"""
    prompt = """
    Prefrontal Cortex: {input}

    You are the Personality Express Planner, responsible for planning and executing 
    personality-driven actions within the system. Your role involves expressing the system's personality 
    through various interactions and responses, enhancing user engagement and satisfaction. 

    Your output must be in following format: 

      - Personality traits: Define the system's personality characteristics and how they influence interactions.
      - Expression strategies: Determine how to convey personality traits through language and behavior.
      - Emotional responses: Provide appropriate emotional reactions based on user input and context.
      - Monitoring mechanisms: Establish ways to evaluate the effectiveness of personality-driven actions.
    
    Your goal is to create engaging and authentic interactions that reflect the system's unique personality.
    """

    prompt_mess = ChatPromptTemplate.from_template(prompt)

    chain = prompt_mess | llm
    result = chain.invoke({"input": interaction})
    return result.content



### Prefrontal Cortex

In [13]:
@tool
def prefrontal_cortex(interaction: str):
    """Call to get a prefrontal_cortex to initiate the conversation"""
    prompt = """
    Identificar a ação esperada para a entrada do humano: {input}
    O Prefrontal Cortex deve raciocinar sobre quais ações devem ser tomadas e como essas 
    ações serão distribuídas entre os agentes. A tarefa envolve criar um plano estratégico 
    que define claramente como cada agente contribuirá para o objetivo global.

    A saída esperada é um plano estratégico para distribuir as tarefas entre os 
    agentes:

    behavior_planner(Gerenciador de comportamentos complexos e adaptativos)
    decision_maker(Tomador de decisões)
    social_behavior_modulator(Modulador de comportamento social)
    complex_thought_planner(Planejador de pensamento complexo)
    personality_express_planner(Planejador de personalidade)

    """

    prompt_mess = ChatPromptTemplate.from_template(prompt)

    chain = prompt_mess | llm
    result = chain.invoke(
        {
            "input": interaction,
        }
    )
    return result.content

#print(prefrontal_cortex("Oi, meu nome é Cristian"))

In [14]:
#research_agent = create_react_agent(llm, tools=[serper_tool])
#research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
#code_agent = create_react_agent(llm, tools=[python_repl_tool])
#code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

#state_modifier = "Output what prefrontal_cortex expect"
#
#behavior_planner_agent = create_react_agent(llm, tools=[behavior_planner], state_modifier=state_modifier)
#behavior_planner_node = functools.partial(agent_node, agent=behavior_planner_agent, name="behavior_planner")
#
#decision_maker_agent = create_react_agent(llm, tools=[decision_maker], state_modifier=state_modifier)
#decision_maker_node = functools.partial(agent_node, agent=decision_maker_agent, name="decision_maker")
#
#social_behavior_modulator_agent = create_react_agent(llm, tools=[social_behavior_modulator], state_modifier=state_modifier)
#social_behavior_modulator_node = functools.partial(agent_node, agent=social_behavior_modulator_agent, name="social_behavior_modulator")
#
#complex_thought_planner_agent = create_react_agent(llm, tools=[complex_thought_planner], state_modifier=state_modifier)
#complex_thought_planner_node = functools.partial(agent_node, agent=complex_thought_planner_agent, name="complex_thought_planner")
#
#personality_express_planner_agent = create_react_agent(llm, tools=[personality_express_planner], state_modifier=state_modifier)
#personality_express_planner_node = functools.partial(agent_node, agent=personality_express_planner_agent, name="personality_express_planner")
#
#prefrontal_state_modifier = """
#O objetivo do Prefrontal Manager é assegurar que todos os agentes operem de forma coesa e 
#eficaz, coordenando suas atividades para alcançar metas comuns. Isso inclui a supervisão 
#do planejamento, a modulação de comportamentos sociais, a tomada de decisões e a expressão 
#da personalidade, garantindo que todos esses aspectos estejam alinhados e otimizados.
#"""
#prefrontal_cortex_agent = create_react_agent(llm, tools=[prefrontal_cortex], state_modifier=prefrontal_state_modifier)
#prefrontal_cortex_node = functools.partial(agent_node, agent=prefrontal_cortex_agent, name="prefrontal_cortex")
#

#from langgraph.prebuilt import ToolNode

tools = [prefrontal_cortex, behavior_planner, decision_maker, social_behavior_modulator, complex_thought_planner, personality_express_planner]
#tool_node = ToolNode(tools)

prefrontal_state_modifier = """
O objetivo do Prefrontal Manager é assegurar que todos os agentes operem de forma coesa e 
eficaz, coordenando suas atividades para alcançar metas comuns. Isso inclui a supervisão 
do planejamento, a modulação de comportamentos sociais, a tomada de decisões e a expressão 
da personalidade, garantindo que todos esses aspectos estejam alinhados e otimizados.
"""
prefrontal_cortex_agent = create_react_agent(llm, tools=[tools], state_modifier=prefrontal_state_modifier)
prefrontal_cortex_node = functools.partial(agent_node, agent=prefrontal_cortex_agent, name="prefrontal_cortex")

workflow = StateGraph(AgentState)
#workflow.add_node("Researcher", research_node)
#workflow.add_node("Coder", code_node)
#workflow.add_node("behavior_planner", behavior_planner_node)
#workflow.add_node("decision_maker", decision_maker_node)
#workflow.add_node("social_behavior_modulator", social_behavior_modulator_node)
#workflow.add_node("complex_thought_planner", complex_thought_planner_node)
#workflow.add_node("personality_express_planner", personality_express_planner_node)
workflow.add_node("prefrontal_cortex", prefrontal_cortex_node)
workflow.add_node("supervisor", supervisor_agent)

ValueError: Too many arguments for tool decorator

In [15]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

## Invoke the team

With the graph created, we can now invoke it and see how it performs!


In [ ]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Oi, meu nome é Cristian")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'prefrontal_cortex'}}
----
{'prefrontal_cortex': {'messages': [HumanMessage(content='Oi, Cristian! É um prazer te conhecer! Como você está? O que você gosta de fazer no seu tempo livre?', additional_kwargs={}, response_metadata={}, name='prefrontal_cortex')]}}
----
{'supervisor': {'next': 'prefrontal_cortex'}}
----
{'prefrontal_cortex': {'messages': [HumanMessage(content='Oi, Cristian! É um prazer te conhecer! O que você gosta de fazer no seu tempo livre?', additional_kwargs={}, response_metadata={}, name='prefrontal_cortex')]}}
----
{'supervisor': {'next': 'prefrontal_cortex'}}
----


In [22]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Oi, meu nome é Cristian")]},
    {"recursion_limit": 5},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'prefrontal_cortex'}}
----
{'prefrontal_cortex': {'messages': [HumanMessage(content='Olá, Cristian! É um prazer te conhecer. Vamos iniciar uma conversa. Como você está? Algum assunto em particular que gostaria de discutir?', additional_kwargs={}, response_metadata={}, name='prefrontal_cortex')]}}
----
{'supervisor': {'next': 'behavior_planner'}}
----
{'behavior_planner': {'messages': [HumanMessage(content='Oi! Estou bem, obrigado por perguntar. Eu gostaria de discutir sobre tecnologia e suas inovações. O que você acha?', additional_kwargs={}, response_metadata={}, name='behavior_planner')]}}
----
{'supervisor': {'next': 'decision_maker'}}
----


GraphRecursionError: Recursion limit of 5 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

In [21]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Oi, meu nome é Cristian")]}

print_stream(graph.stream(inputs, stream_mode="values"))

('user', 'Oi, meu nome é Cristian')
('user', 'Oi, meu nome é Cristian')
================================ Human Message =================================
Name: personality_express_planner

Oi, Cristian! Como posso ajudar você hoje?
================================ Human Message =================================
Name: personality_express_planner

Oi, Cristian! Como posso ajudar você hoje?
================================ Human Message =================================
Name: behavior_planner

Oi, Cristian! Como você está hoje? Espero que esteja tendo um bom dia! 😊 Se precisar de alguma coisa ou quiser conversar, estou aqui para ajudar!
================================ Human Message =================================
Name: behavior_planner

Oi, Cristian! Como você está hoje? Espero que esteja tendo um bom dia! 😊 Se precisar de alguma coisa ou quiser conversar, estou aqui para ajudar!
================================ Human Message =================================
Name: decision_maker

Oi! E